<a href="https://colab.research.google.com/github/pren1/VAST/blob/master/VAST_data_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# We take a deep look into the data we have

In [0]:
!rm -rf VAST/
!git clone https://github.com/pren1/VAST.git

Cloning into 'VAST'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 76 (delta 32), reused 51 (delta 18), pack-reused 0
Unpacking objects: 100% (76/76), done.


In [0]:
!pip3 install soundfile
!pip3 install librosa

In [0]:
from VAST.util import *
import pdb
import pprint
from tqdm import tqdm
import numpy as np
import IPython
import soundfile as sf
import matplotlib.pyplot as plt
import librosa

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
vtb_audio = "gdrive/My Drive/VAST/"
# vtb_audio = "/Volumes/GoogleDrive/我的云端硬盘/VAST/"

In [0]:
def get_all_sub_folders(path):
  whole_path_set = []
  corresponding_label = []
  first_level_folders = [f.path for f in os.scandir(path) if f.is_dir()]
  for label_index, first in tqdm(enumerate(first_level_folders)):
    second_level_folders = [f.path for f in os.scandir(first) if f.is_dir()]
    # pprint.pprint(second_level_folders)
    for second in second_level_folders:
      all_sub_files = files_in_target_folder(second, 'flac')
      'build every paths'
      full_sub_files = [second + "/" + x for x in all_sub_files]
      # pprint.pprint(full_sub_files)
      whole_path_set.extend(full_sub_files)
      corresponding_label.extend([label_index] * len(full_sub_files))
  assert len(whole_path_set) == len(corresponding_label), "logic error"
  return whole_path_set, corresponding_label

def obtain_name_dict_for_debug(path):
  first_level_folders = [f.path.split('/')[-1] for f in os.scandir(path) if f.is_dir()]
  return first_level_folders

def rectify_according_to_variance(splitted_voice, std_window_size=960, post_process_clip_threshold = 0.007):
    std_array = rolling_window(splitted_voice, std_window_size)
    end_pass, end_clip_index = end_clip(std_array, threshold = post_process_clip_threshold)
    front_pass, front_clip_index = front_clip(std_array, threshold = post_process_clip_threshold)
    if end_pass and front_pass:
      assert front_clip_index < len(splitted_voice) - end_clip_index, "STD array incorrect"
      clipped_splitted_voice = splitted_voice[front_clip_index:~end_clip_index] 
      return clipped_splitted_voice
    else:
      return []  

def clip_one_second_data(seq_data, samplerate, resulted_rate = 16000):
  if len(seq_data) < samplerate:
    print("Insufficient length")
    return []
  'First, let us down sample'
  y_16k = librosa.resample(seq_data, samplerate, resulted_rate)
  clipped_res = []
  'Then, clip it'
  for index in range(0, len(y_16k), resulted_rate):
    'make sure you get the 1 seconds clipped data'
    current_clip = y_16k[index:index + resulted_rate]
    if len(current_clip) == resulted_rate:
      clipped_res.append(current_clip)
  return clipped_res

def Show_random_pairs(data_path, label, k = 10):
  order = np.arange(len(data_path))
  np.random.shuffle(order)
  idx = order[:k]

  whole_data = []
  whole_label = []

  for single_idx in tqdm(idx):
    single_data_path = data_path[single_idx]
    single_label = label[single_idx]  
    data, samplerate = sf.read(single_data_path)
    if len(data) > 0:
      clipped_data = clip_one_second_data(data, samplerate, 8000)
      corresponding_label = [single_label] * len(clipped_data)
      if len(clipped_data) > 0:
        whole_data.extend(clipped_data)
        whole_label.extend(corresponding_label)
        # for single in clipped_data:
        #   plt.plot(single, color = 'orange', label='read in sample data')
        #   plt.title(f"Label: {single_label}")
        #   plt.show()
        #   IPython.display.display(IPython.display.Audio(np.asarray(single).T, rate=8000))
  assert len(whole_data) == len(whole_label)
  return whole_data, whole_label    

In [0]:
data_path, label = get_all_sub_folders(vtb_audio)
name_list = obtain_name_dict_for_debug(vtb_audio)
pprint.pprint(name_list)

4it [00:00,  4.48it/s]

['宝鐘マリン', '白上フブキ', '天音かなた', '夏色まつり']


In [0]:
# data, label = Show_random_pairs(data_path, label, k=len(label))
data, label = Show_random_pairs(data_path, label, k=10000)

  0%|          | 1/10000 [00:02<6:14:26,  2.25s/it]

Insufficient length


  0%|          | 3/10000 [00:07<6:09:38,  2.22s/it]

Insufficient length


  0%|          | 5/10000 [00:13<7:36:42,  2.74s/it]

Insufficient length


  0%|          | 7/10000 [00:23<10:41:00,  3.85s/it]

KeyboardInterrupt: ignored

  0%|          | 7/10000 [00:24<9:52:31,  3.56s/it] 


In [0]:
np.asarray(data).shape

In [0]:
np.asarray(label).shape

In [0]:
save_data_array_as_npy(data, f"{vtb_audio}/one_second_filtered_data")
save_data_array_as_npy(label, f"{vtb_audio}/one_second_filtered_label")

In [0]:
len(data_path)